<a href="https://colab.research.google.com/github/Villanelev/credit_scoring_project/blob/main/notebooks/credit_scoring2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
import os

USER_NAME = "user_name"
REPO_NAME = "repo_name"
EMAIL = "email"
TOKEN = "token"

REPO_URL = f"https://{TOKEN}@github.com/{USER_NAME}/{REPO_NAME}.git"

!git clone {REPO_URL}

%cd {REPO_NAME}

!git config --global user.email "{EMAIL}"
!git config --global user.name "{USER_NAME}"

print("Связь установлена! Репозиторий склонирован.")

Cloning into 'credit_scoring_project'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 2), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 591.34 KiB | 7.58 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/credit_scoring_project/credit_scoring_project
Связь установлена! Репозиторий склонирован.


In [ ]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/home-credit-default-risk.zip'
destination_path = '/content/credit_scoring_project/data'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destination_path)
    print(f"Успех! Данные распакованы в {destination_path}")
else:
    print("Файл не найден на Диске. Проверь, что он залился и называется именно так.")

Mounted at /content/drive
Успех! Данные распакованы в /content/credit_scoring_project/data


# обработка Application

In [ ]:
def Application_preprocessing(path):
    df = pd.read_csv(path)

    time_cols = ['DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_EMPLOYED', 'DAYS_BIRTH']
    clip_cols = ['AMT_INCOME_TOTAL', 'AMT_ANNUITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
                 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

    # Инфа про жилье
    enviroment_info = [i for i in df.columns if '_MEDI' in i]

    # 1. Исправляем время (делаем положительным)
    for col in time_cols:
        if col in df.columns:
            df[col] = df[col].abs()

    # bad_col = [col for col in df.columns if df[col].value_counts(normalize=True).max() > 0.99]
    # work_df = df.drop(columns=bad_col).copy()
    work_df = df.copy()

    if 'DAYS_EMPLOYED' in work_df.columns:
        work_df['DAYS_EMPLOYED'] = work_df['DAYS_EMPLOYED'].replace(365243, np.nan)

    dublicates_col = [x for x in work_df.columns if any(m in x for m in ['_AVG', '_MODE'])]
    work_df.drop(columns=dublicates_col, inplace=True)

    for col in enviroment_info:
        if col in work_df.columns:
            work_df[col] = work_df[col].fillna(0)

    cat_cols = work_df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        work_df[col] = work_df[col].fillna("Unknown")

    for col in clip_cols:
        if col in work_df.columns:
            limit = work_df[col].quantile(0.995)
            work_df[col] = work_df[col].clip(upper=limit)

    work_df['ANNUITY_INCOME_RATIO'] = work_df['AMT_ANNUITY'] / (work_df['AMT_INCOME_TOTAL'] + 1)

    work_df['CREDIT_INCOME_RATIO'] = work_df['AMT_CREDIT'] / (work_df['AMT_INCOME_TOTAL'] + 1)

    work_df['CREDIT_TERM'] = work_df['AMT_CREDIT'] / (work_df['AMT_ANNUITY'] + 1)

    if 'DAYS_EMPLOYED' in df.columns and 'DAYS_BIRTH' in df.columns:
        df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 1)

    work_df['EXT_SOURCES_PROD'] = work_df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    work_df['EXT_SOURCES_MEAN'] = work_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    work_df['EXT_SOURCES_MIN'] = work_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

    return work_df

# добавка Bureau


In [ ]:
def process_and_merge_bureau(path, curr_df):
    bureau = pd.read_csv(path)

    for col in ['DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT']:
        bureau[col] = bureau[col].replace(365243, np.nan)

    closed_mask = bureau['CREDIT_ACTIVE'].isin(['Closed', 'Sold'])
    bureau.loc[closed_mask, 'AMT_CREDIT_SUM_DEBT'] = bureau.loc[closed_mask, 'AMT_CREDIT_SUM_DEBT'].fillna(0)

    bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
    bureau['AMT_CREDIT_SUM_LIMIT'] = bureau['AMT_CREDIT_SUM_LIMIT'].fillna(0)

    bureau['AMT_DEBT_IS_NAN'] = bureau['AMT_CREDIT_SUM_DEBT'].isna().astype(int)

    bureau = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE', 'CREDIT_TYPE', 'CREDIT_CURRENCY'], dummy_na=False)

    bureau['DEBT_CREDIT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM'] + 1.0)

    agg_dict = {
        # Даты
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['max'],

        # Деньги
        'AMT_CREDIT_SUM': ['max', 'sum', 'mean'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'sum', 'mean'],
        'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_SUM_LIMIT': ['sum', 'mean'],

        # Дисциплина
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DEBT_CREDIT_RATIO': ['mean', 'max'],
        'AMT_DEBT_IS_NAN': ['mean'], # Какая доля кредитов с неизвестным долгом

        # Категории (OHE) - считаем количество (sum) и долю (mean)
        'CREDIT_ACTIVE_Active': ['sum', 'mean'],
        'CREDIT_ACTIVE_Closed': ['sum'],
        'CREDIT_TYPE_Credit card': ['sum'],
        'CREDIT_TYPE_Microloan': ['sum'],
    }

    # Группировка
    bureau_agg = bureau.groupby('SK_ID_CURR').agg(agg_dict)

    # Плоские названия колонок (BUREAU_DAYS_CREDIT_MIN и т.д.)
    bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
    bureau_agg = bureau_agg.reset_index()

    work_df = curr_df.merge(bureau_agg, on='SK_ID_CURR', how='left')
    bureau_cols = [c for c in work_df.columns if c.startswith('BUREAU_')]

    fill_zero_keywords = ['_SUM', '_CNT', '_COUNT', '_ACTIVE_', '_CLOSED_', '_MICROLOAN_', '_OVERDUE_']
    cols_to_fill_zero = [c for c in bureau_cols if any(k in c for k in fill_zero_keywords)]

    cols_to_fill_zero.extend([c for c in bureau_cols if 'AMT_DEBT_IS_NAN' in c])

    work_df[cols_to_fill_zero] = work_df[cols_to_fill_zero].fillna(0)

    return work_df

# добавка Installments


In [ ]:
def process_and_merge_installments(path_to_file, current_df):

    ins = pd.read_csv(path_to_file)

    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']

    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)

    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

    # Словарь агрегаций
    agg_dict = {
        'NUM_INSTALMENT_VERSION': ['nunique'], # Сколько раз менялись условия
        'DPD': ['max', 'mean', 'sum'],         # Макс. просрочка, средняя, общая (дни)
        'DBD': ['max', 'mean', 'sum'],         # Макс. досрочность (дни)
        'PAYMENT_PERC': ['max', 'mean', 'var'],# Дисциплина оплаты (вариация важна!)
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'], # Суммы недоплат
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],      # Сколько должен был платить
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],  # Сколько реально платил
        'DAYS_ENTRY_PAYMENT': ['max', 'min']           # Когда был последний и первый платеж
    }

    ins_agg = ins.groupby('SK_ID_CURR').agg(agg_dict)

    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

    ins_agg = ins_agg.reset_index()


    work_df = current_df.merge(ins_agg, on='SK_ID_CURR', how='left')

    new_cols = [c for c in work_df.columns if c.startswith('INSTAL_')]

    fill_zero_cols = [c for c in new_cols if any(x in c for x in ['_SUM', '_COUNT', '_MAX', '_MIN', 'DPD', 'DBD', 'NUM_INSTALMENT_VERSION'])]

    work_df[fill_zero_cols] = work_df[fill_zero_cols].fillna(0)

    return work_df


# добавляем previous_application


In [ ]:
def process_previous_application(path_to_file, current_df):

    prev = pd.read_csv(path_to_file)

    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)


    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

    prev = pd.get_dummies(prev, columns=['NAME_CONTRACT_STATUS', 'NAME_CLIENT_TYPE', 'NAME_YIELD_GROUP'], dummy_na=False)

    agg_dict = {
        # Сколько просил и сколько дали
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'], # Первоначальный взнос
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],

        # Даты (как давно обращался)
        'DAYS_DECISION': ['min', 'max', 'mean'], # Сколько дней назад было решение
        'CNT_PAYMENT': ['mean', 'sum'],          # На какой срок обычно берет (мес)

        # Статусы (самое важное!)
        'NAME_CONTRACT_STATUS_Approved': ['sum', 'mean'], # Сколько одобрено и доля одобрений
        'NAME_CONTRACT_STATUS_Refused': ['sum', 'mean'],  # Сколько отказов и доля отказов
        'NAME_YIELD_GROUP_high': ['sum'],                 # Сколько раз брал под высокий процент
        'NAME_YIELD_GROUP_low_normal': ['sum'],           # Сколько раз брал под низкий процент
    }

    prev_agg = prev.groupby('SK_ID_CURR').agg(agg_dict)

    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    prev_agg['PREV_APP_COUNT'] = prev.groupby('SK_ID_CURR').size()

    prev_agg = prev_agg.reset_index()

    work_df = current_df.merge(prev_agg, on='SK_ID_CURR', how='left')

    new_cols = [c for c in work_df.columns if c.startswith('PREV_')]

    fill_zero_cols = [c for c in new_cols if any(x in c for x in ['_SUM', '_COUNT', 'Approved', 'Refused', 'high', 'low'])]
    work_df[fill_zero_cols] = work_df[fill_zero_cols].fillna(0)

    cat_cols = work_df.select_dtypes(include=['object']).columns
    for col in cat_cols:
      work_df[col] = work_df[col].fillna("Unknown")

    return work_df

# модель

In [ ]:
df_1 = Application_preprocessing('/content/credit_scoring_project/data/application_train.csv')


In [ ]:
df_2 = process_and_merge_bureau('/content/credit_scoring_project/data/bureau.csv', df_1)
df_3 = process_and_merge_installments('/content/credit_scoring_project/data/installments_payments.csv', df_2)
df_4 = process_previous_application('/content/credit_scoring_project/data/previous_application.csv', df_3)


/tmp/ipython-input-587490211.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
/tmp/ipython-input-587490211.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

## catboost

In [ ]:
!pip install CatBoost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [17]:
X = df_4.drop(columns=['TARGET', 'SK_ID_CURR'])
y = df_4['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = CatBoostClassifier(
    iterations=5100,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    eval_metric='AUC',
    verbose=100,
    #early_stopping_rounds=200,
    task_type='GPU'
)
cat_features = df_4.select_dtypes(include=['object']).columns.tolist()

model.fit(X, y, cat_features=cat_features)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 84ms	remaining: 7m 8s
100:	total: 5.55s	remaining: 4m 34s
200:	total: 9.87s	remaining: 4m
300:	total: 14.5s	remaining: 3m 50s
400:	total: 19.8s	remaining: 3m 51s
500:	total: 24s	remaining: 3m 40s
600:	total: 30.2s	remaining: 3m 46s
700:	total: 34.5s	remaining: 3m 36s
800:	total: 38.8s	remaining: 3m 28s
900:	total: 44.5s	remaining: 3m 27s
1000:	total: 48.8s	remaining: 3m 19s
1100:	total: 53s	remaining: 3m 12s
1200:	total: 58.8s	remaining: 3m 10s
1300:	total: 1m 3s	remaining: 3m 5s
1400:	total: 1m 8s	remaining: 3m 1s
1500:	total: 1m 14s	remaining: 2m 57s
1600:	total: 1m 18s	remaining: 2m 51s
1700:	total: 1m 23s	remaining: 2m 46s
1800:	total: 1m 27s	remaining: 2m 41s
1900:	total: 1m 32s	remaining: 2m 35s
2000:	total: 1m 37s	remaining: 2m 30s
2100:	total: 1m 42s	remaining: 2m 26s
2200:	total: 1m 46s	remaining: 2m 20s
2300:	total: 1m 52s	remaining: 2m 16s
2400:	total: 1m 56s	remaining: 2m 11s
2500:	total: 2m	remaining: 2m 5s
2600:	total: 2m 6s	remaining: 2m 1s
2700:	total: 2m 10s	

In [ ]:
# Создаем таблицу: название колонки - её важность
features_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.get_feature_importance()
}).sort_values(by='importance', ascending=False)

print(features_df.head(15))

                                    feature  importance
91                         EXT_SOURCES_MEAN   13.416745
89                              CREDIT_TERM    3.322986
92                          EXT_SOURCES_MIN    2.893849
111            BUREAU_DEBT_CREDIT_RATIO_MAX    2.503125
15                               DAYS_BIRTH    2.384032
40                             EXT_SOURCE_2    1.998802
41                             EXT_SOURCE_3    1.964772
161                   PREV_CNT_PAYMENT_MEAN    1.621816
120                         INSTAL_DPD_MEAN    1.612838
8                           AMT_GOODS_PRICE    1.496007
7                               AMT_ANNUITY    1.492709
16                            DAYS_EMPLOYED    1.449616
139           INSTAL_DAYS_ENTRY_PAYMENT_MAX    1.426078
166  PREV_NAME_CONTRACT_STATUS_Refused_MEAN    1.381199
97           BUREAU_DAYS_CREDIT_ENDDATE_MAX    1.316516


## extreme

In [ ]:
!pip install xgboost -q

# сабмит на кегль


In [18]:
df_test_1 = Application_preprocessing('/content/credit_scoring_project/data/application_test.csv')

print("Джойним бюро...")
df_test_2 = process_and_merge_bureau('/content/credit_scoring_project/data/bureau.csv', df_test_1)

print("Джойним платежи...")
df_test_3 = process_and_merge_installments('/content/credit_scoring_project/data/installments_payments.csv', df_test_2)

print("Джойним предыдущие заявки...")
df_test_final = process_previous_application('/content/credit_scoring_project/data/previous_application.csv', df_test_3)


submission_ids = df_test_final['SK_ID_CURR']

X_test_final = df_test_final.copy()

X_test_final = X_test_final[X_train.columns]


print("Запуск предсказания...")
test_preds = model.predict_proba(X_test_final)[:, 1]

submission = pd.DataFrame({
    'SK_ID_CURR': submission_ids,
    'TARGET': test_preds
})

submission.to_csv('submission_catboost.csv', index=False)
print("Файл submission_catboost.csv успешно создан!")

Джойним бюро...
Джойним платежи...
Джойним предыдущие заявки...


/tmp/ipython-input-587490211.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
/tmp/ipython-input-587490211.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

Запуск предсказания...
Файл submission_catboost.csv успешно создан!


In [22]:
model.save_model('model_1_catboost.cbm')

!git add .
!git commit -m "CatBoost app+bur+inst+prev"
!git push


[main 7ac3f9e] CatBoost app+bur+inst+prev
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 model_1_catboost.cbm
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.28 MiB | 3.07 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Villanelev/credit_scoring_project.git
   a6ec97f..7ac3f9e  main -> main
